# Pregunta 5 
Considerando el `pagelink_sample.csv` representar como un grafo en Spark los contenidos de wikipedia (considerando los contenidos como nodos y los links como aristas) como una lista de adyacencia y mostrar un recorrido en la estructura.

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [2]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

sqlContext = SQLContext(sc)

21/10/31 17:27:39 WARN Utils: Your hostname, Ideapad3 resolves to a loopback address: 127.0.1.1; using 192.168.0.237 instead (on interface wlp0s20f3)
21/10/31 17:27:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/10/31 17:27:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/home/luis/anaconda3/envs/organizacionDatos/lib/python3.9/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
contents = sqlContext.read.csv('./Wikipedia_dataset_2c2021/contents.csv', 
                                    header=True, inferSchema=True, multiLine=True)

rdd_contents = contents.rdd

pagelinks_sample = sqlContext.read.csv('./Wikipedia_dataset_2c2021/pagelinks_sample.csv', 
                                    header=True, inferSchema=True, multiLine=True)
rdd_pagelinks_sample = pagelinks_sample.rdd

## Busco RDD del estilo (K,V)

Extraigo la informacion que necesito de cada dataset y lo expreso en tuplas clave valor.

In [4]:
rdd_contents = rdd_contents.map(lambda x: (x.title, x.id))

In [5]:
rdd_pagelinks_sample = rdd_pagelinks_sample.map(lambda x: (x.pl_title, x.pl_from) )

## Trabajo con los contenidos que estan en contents.


Intersecto los titulos que estan tanto en `pagelinks_sample.csv` como en `contents.csv`

rdd_wiki tendra tuplas de la forma `(titulo, id_contenido_que_lo_referencia, id_contenido)` \
o equivalentemente `(pl_title, pl_from, id)`

In [6]:
rdd_wiki = rdd_pagelinks_sample.join(rdd_contents)

## Agrupo por nodo los distintos enlaces internos que tienen

Obtengo una lista adyacente del estilo 

`[ (Nodo_1, [ (titulo_1,id_1), (titulo_2,id_2) ...  ]) , (Nodo_2, [ ... ]) ]`

Donde (titulo,id) representa cada arista que tiene el nodo.

In [7]:
lista_adyacente = rdd_wiki.map(lambda x: ( x[1][0], (x[0],x[1][1]) ) )\
                    .groupByKey().map(lambda x: (x[0], list(x[1])))

# Muestro algunos elementos de la lista de Adyacencia

In [8]:
lista_adyacente.take(10)

[(9634840,
  [('Interdisciplinariedad', '836987'),
   ('Dibujo', '1574475'),
   ('Londres', '10256'),
   ('Oporto', '55368'),
   ('Audiovisual', '67733'),
   ('Investigador', '414362'),
   ('1976', '5234'),
   ('ORCID', '7229853'),
   ('PALOP', '341986')]),
 (5843508,
  [('2010', '21612'),
   ('Chileno', '58872'),
   ('Soccerway', '9468272'),
   ('Wikidata', '5291797'),
   ('2011', '866581'),
   ('Metro', '1784'),
   ('1985', '3080'),
   ('2018', '2374070'),
   ('Chile', '276780')]),
 (3391328,
  [('2010', '21612'), ('Raystorm', '1572894'), ('Coldplay', '3344550')]),
 (4999398,
  [('2010', '21612'),
   ('Stanitsa', '351982'),
   ('Msnm', '8470660'),
   ('1937', '9979'),
   ('Volgogrado', '40631'),
   ('1979', '3443'),
   ('Krasnodar', '429377'),
   ('Wikidata', '5291797'),
   ('Rusia', '2400685'),
   ('1934', '7146'),
   ('1970', '5240'),
   ('Altitud', '30455'),
   ('1953', '7718'),
   ('1966', '5278'),
   ('Rostov', '688959'),
   ('1989', '5205'),
   ('1963', '5281'),
   ('OKATO', '5

# Recorro la lista de adyacencia

Para recorrer el grafo tomaremos un arista cualquiera. 

Cada nodo tiene una lista de aristas, por ende me voy por la arista `n = i % cant_aristas`, \
tal que el ID lo obtengo como `nodo_hijo = lista_adyacente[1][n][1]`

In [12]:
nodo_origen = lista_adyacente.first()

for i in range(0,5):
    
    id_nodo_origen = nodo_origen[0]
    cant_aristas = len(nodo_origen[1])
    
    id_nodo_hijo = int(nodo_origen[1][i%cant_aristas][1])
    titulo_nodo_hijo = nodo_origen[1][i%cant_aristas][0]
    
    nodo_hijo = lista_adyacente.filter(lambda x: x[0] == id_nodo_hijo).first()
    
    print(
    """Estamos el nodo: {nodo_orig}, y vamos al nodo {nodo_hijo} de titulo: {titulo}""".format(
        nodo_orig=id_nodo_origen, nodo_hijo=id_nodo_hijo, titulo=titulo_nodo_hijo ) 
    )
    
    nodo_origen = nodo_hijo

Estamos el nodo: 705024, y vamos al nodo 1614013 de titulo: Chessman


Estamos el nodo: 1614013, y vamos al nodo 4695251 de titulo: Konnan


Estamos el nodo: 4695251, y vamos al nodo 866572 de titulo: 2012


Estamos el nodo: 866572, y vamos al nodo 6396 de titulo: Portugal


Estamos el nodo: 6396, y vamos al nodo 76158 de titulo: Espiritismo


# Conclusion

Logramos representar cada nodo como una tupla  `(Nodo, [ Lista de vecinos ])`, esto representa un nodo del grafo junto con sus distintas aristas vecinas.

Luego para recorrerla, seguimos un camino aleatorio, es decir, al pararnos en un nodo con `cantidad_aristas` finita y nos vamos por \
la arista `n = i % cantidad_aristas`.

Este recorrido tiene como ventaja evitar entrar en un ciclo infinito en caso de que la primera arista sea si misma, porque en la siguiente iteracion pasa por la arista que le sigue.